In [1]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("nbastats.db")
cur = conn.cursor()

#print the tables in the database
for row in cur.execute("SELECT name FROM sqlite_master WHERE type='table';"):
    print(row)


('advanced_stats',)
('allgames_stats',)
('allstar_games_stats',)
('game_highs_stats',)
('per_game_stats',)
('salaries',)
('totals_stats',)


In [2]:
#Find how many playoff games each player has played in their career
cur.execute('''
            SELECT Player, COUNT(G) FROM per_game_stats
            WHERE RSorPO = 'Playoffs'
            GROUP BY Player;
            ''')

df = pd.DataFrame(cur.fetchall(), columns = ['Player','PO Games Played'])
df

,Player,PO Games Played
0,Kobe Bryant,15
1,Lebron James,13
2,Michael Jordan,13


In [3]:
#Retrieve the 10 highest scoring average in the playoffs
cur.execute('''
            SELECT Player, PTS FROM per_game_stats
            WHERE RSorPO = 'Playoffs'
            ORDER BY PTS DESC
            LIMIT 10;
            ''')

df = pd.DataFrame(cur.fetchall(), columns = ['Player','PTS'])
df


,Player,PTS
0,Michael Jordan,43.7
1,Michael Jordan,36.7
2,Michael Jordan,36.3
3,Michael Jordan,35.7
4,Lebron James,35.3
5,Michael Jordan,35.1
6,Michael Jordan,34.8
7,Michael Jordan,34.5
8,Lebron James,34.0
9,Lebron James,32.8


In [4]:
#Retreive the avg at the highest point scored in the playoffs from MJ
cur.execute('''
            SELECT PGS.Player, PGS.PTS, PGS.G, TS.PTS FROM per_game_stats PGS
            LEFT JOIN totals_stats TS
            on PGS.Season = TS.Season 
               AND PGS.RSorPO = TS.RSorPO 
               AND PGS.Player = TS.Player
            WHERE TS.PTS = (SELECT MAX(PTS) FROM totals_stats
                            WHERE RSorPO='Playoffs' AND Player = 'Michael Jordan');
            ''')

df = pd.DataFrame(cur.fetchall(), columns = ['Player','PTS','PO Games Played','Play Scoring Total'])
df

,Player,PTS,PO Games Played,Play Scoring Total
0,Michael Jordan,34.5,22,759


In [5]:
#Retreive the avg at the highest point scored in the playoffs from Kobe
cur.execute('''
            SELECT PGS.Player, PGS.PTS, PGS.G, TS.PTS FROM per_game_stats PGS
            LEFT JOIN totals_stats TS
            on PGS.Season = TS.Season 
               AND PGS.RSorPO = TS.RSorPO 
               AND PGS.Player = TS.Player
            WHERE TS.PTS = (SELECT MAX(PTS) FROM totals_stats
                            WHERE RSorPO='Playoffs' AND Player = 'Kobe Bryant');
            ''')

df = pd.DataFrame(cur.fetchall(), columns = ['Player','PTS','PO Games Played','Play Scoring Total'])
df

,Player,PTS,PO Games Played,Play Scoring Total
0,Kobe Bryant,30.2,23,695


In [6]:
#Retreive the avg at the highest point scored in the playoffs from Lebron
cur.execute('''
            SELECT PGS.Player, PGS.PTS, PGS.G, TS.PTS FROM per_game_stats PGS
            LEFT JOIN totals_stats TS
            on PGS.Season = TS.Season 
               AND PGS.RSorPO = TS.RSorPO 
               AND PGS.Player = TS.Player
            WHERE TS.PTS = (SELECT MAX(PTS) FROM totals_stats
                            WHERE RSorPO='Playoffs' AND Player = 'Lebron James');
            ''')

df = pd.DataFrame(cur.fetchall(), columns = ['Player','PTS','PO Games Played','Play Scoring Total'])
df

,Player,PTS,PO Games Played,Play Scoring Total
0,Lebron James,34,22,748


In [7]:
#Find the total playoff stats of each player
cur.execute('''
            SELECT Player, SUM(G), SUM(PTS), SUM(TRB), SUM(AST), SUM(STL), SUM(BLK), SUM(TOV), SUM("3P")
            FROM totals_stats
            WHERE RSorPO = 'Playoffs'
            GROUP BY Player
            ORDER BY SUM(G) DESC;
            ''')

df = pd.DataFrame(cur.fetchall(), columns = ['Player',' Total PO Games Played','Total PO Points',
                                             'Total PO Rebounds','Total PO Assits','Total PO Steals', 
                                             'Total PO Blocks', 'Total PO Turnovers','Total PO 3P made'])
df

,Player,Total PO Games Played,Total PO Points,Total PO Rebounds,Total PO Assits,Total PO Steals,Total PO Blocks,Total PO Turnovers,Total PO 3P made
0,Lebron James,239,6911,2122,1687,419,232,866,370
1,Kobe Bryant,220,5640,1119,1040,310,144,647,292
2,Michael Jordan,179,5987,1152,1022,376,158,546,148


In [8]:
#Find the avg points scored in the playoffs by each player
cur.execute('''
            SELECT Player, ROUND((CAST(SUM(PTS) as float)/SUM(G)),1) FROM totals_stats
            WHERE RSorPO = 'Playoffs'
            GROUP BY Player
            ORDER BY 2 DESC;
            ''')

df = pd.DataFrame(cur.fetchall(), columns = ['Player','Playoff Average'])
df

,Player,Playoff Average
0,Michael Jordan,33.4
1,Lebron James,28.9
2,Kobe Bryant,25.6


In [9]:
#close connection
conn.close()